# Test
Apply the model to the test dataset

In [1]:
import argparse

parser = argparse.ArgumentParser(description='description')
parser.add_argument('-in', '--input', type=str, default='dataset_test.txt')
parser.add_argument('-out', '--output', type=str, default='prob_test.root')
parser.add_argument('-n', '--nsize', type=int, default=-1)
args = parser.parse_args(args=[])
#args = parser.parse_args()

filename_input = args.input
filename_output = args.output
nevt = args.nsize



In [2]:
import pandas as pd

nevt = None if nevt < 0 else nevt
dataset = pd.read_csv(filename_input, skipinitialspace=True, nrows=nevt)
dataset

,EvtNo,ID,Shift,Sigma,Time,Time0,Time1,Time2,Time3,Time4,Time5,Time6,Time7,Time8,Time9,Time10,Time11,Time12,Time13,Time14
0,0,0,0.000749,1,5,0.001115,0.000799,0.000261,-0.000152,0.000113,0.000791,0.000960,0.000206,-0.000914,-0.001606,-0.001591,-0.001096,-0.000396,0.000398,0.001113
1,0,0,0.000766,1,6,0.000781,0.000244,-0.000169,0.000095,0.000774,0.000942,0.000189,-0.000932,-0.001623,-0.001608,-0.001113,-0.000413,0.000381,0.001096,0.001355
2,0,0,0.000785,1,7,0.000225,-0.000188,0.000077,0.000755,0.000924,0.000170,-0.000951,-0.001642,-0.001627,-0.001131,-0.000432,0.000362,0.001077,0.001336,0.001044
3,0,0,0.001585,1,13,-0.001750,-0.002442,-0.002426,-0.001931,-0.001231,-0.000437,0.000278,0.000537,0.000245,0.000108,0.000941,0.002315,0.002802,0.001968,0.001024
4,0,0,0.001756,1,14,-0.002614,-0.002598,-0.002103,-0.001403,-0.000609,0.000106,0.000365,0.000073,-0.000064,0.000769,0.002143,0.002630,0.001796,0.000852,0.000658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586929,4999,0,0.001989,1,1004,-0.000597,0.000508,0.000888,0.000574,0.000622,0.001112,0.001038,0.000077,-0.000815,-0.000812,-0.000306,-0.000194,-0.000629,-0.000924,-0.000543
2586930,4999,0,0.002037,1,1005,0.000461,0.000840,0.000526,0.000574,0.001064,0.000990,0.000029,-0.000863,-0.000860,-0.000354,-0.000242,-0.000677,-0.000972,-0.000591,0.000074
2586931,4999,0,0.001654,1,1009,0.001446,0.001373,0.000412,-0.000480,-0.000477,0.000029,0.000141,-0.000294,-0.000589,-0.000208,0.000457,0.000493,-0.000337,-0.001108,-0.000859
2586932,4999,0,0.001561,1,1010,0.001466,0.000505,-0.000388,-0.000385,0.000122,0.000234,-0.000201,-0.000496,-0.000115,0.000550,0.000586,-0.000244,-0.001015,-0.000766,0.000149


In [3]:
ntime = 15
ndim = 1

dataset_feature = dataset.loc[:, 'Time0':'Time%d' % (ntime-1)].values
dataset_label = dataset.loc[:, 'ID'].values

dataset_feature = dataset_feature.reshape(dataset_feature.shape[0], ntime, ndim)

In [4]:
from keras.models import load_model

model = load_model('peak_classification.h5')

In [5]:
predicted_prob = model.predict(dataset_feature, verbose = 2)

80842/80842 - 102s


In [6]:
from ROOT import TFile, TTree
from array import array

fileOut = TFile.Open(filename_output, "recreate")
treeOut = TTree("tmva", "tmva")
prob_rnn = array('d', [-999])
treeOut.Branch("prob_rnn", prob_rnn, "prob_rnn/D")

for prob in predicted_prob:
    prob_rnn[0] = prob
    treeOut.Fill()

fileOut.WriteTObject(treeOut)
fileOut.Close()

Welcome to JupyROOT 6.22/03
